## Ingest pit_stops.json file


In [0]:
%run ../includes/configuration

In [0]:
%run ../includes/common_functions

In [0]:
dbutils.widgets.text('p_data_source', '')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-28')
v_file_date = dbutils.widgets.get('p_file_date')

# Step 1.- Read the JSON file using the spark dataframe reader 

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
pit_stop_schema = StructType(fields=[StructField('raceId', IntegerType(), False),
                                    StructField('driverId', IntegerType(), True),
                                    StructField('stop', StringType(), True),
                                    StructField('lap', IntegerType(), True),
                                    StructField('time', StringType(), True),
                                    StructField('duration', StringType(), True),
                                    StructField('milliseconds', IntegerType(), True)])

In [0]:
pit_stops_df = spark.read \
.schema(pit_stop_schema) \
.option('multiLine', True) \
.json(f'{raw_folder_path}/{v_file_date}/pit_stops.json')

## Step 2.- Rename columns and add new columns 
1. Renamed driverId and raceId
2. Add ingestion_date with current timestamp


In [0]:
from pyspark.sql.functions import col, lit

In [0]:
final_df = add_ingestion_date(pit_stops_df) \
                       .withColumnRenamed('driverId', 'driver_id') \
                       .withColumnRenamed('raceId', 'race_id') \
                       .withColumn('data_source', lit(v_data_source)) \
                       .withColumn('file_date', lit(v_file_date))

## Step 3.- Write to output tp processed container in parquet format

In [0]:
re_arrange_partition_column(final_df,'race_id')

In [0]:
# overwrite_partition(final_df, 'f1_processed', 'pit_stops', 'race_id')

In [0]:
merge_condition = 'tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.stop = src.stop AND tgt.race_id = src.race_id'
merge_delta(final_df, 'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
dbutils.notebook.exit('Success')

In [0]:
%sql select * from f1_processed.pit_stops;